In [1]:
# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [2]:
from torch import Tensor
import torch

import typing
from typing import List, Tuple

In [3]:
import lincoln as lnc
from lincoln.operations import Operation, ParamOperation
from lincoln.layers import Layer
from lincoln.activations import Activation, LinearAct

In [4]:
class Dense(Layer):
    '''
    Once we define all the Operations and the outline of a layer, all that remains to implement here 
    is the _setup_layer function!
    '''
    def __init__(self, 
                 neurons: int, 
                 activation: Activation = LinearAct) -> None:
        super().__init__(neurons)
        self.activation = activation

    def _setup_layer(self, num_in: int) -> None:
        # weights
        self.params.append(torch.empty(num_in, self.neurons).uniform_(-1, 1))
        
        # bias
        self.params.append(torch.empty(1, self.neurons).uniform_(-1, 1))
        
        self.operations = [WeightMultiply(self.params[0]), 
                           BiasAdd(self.params[1])] + [self.activation]

In [5]:
class WeightMultiply(ParamOperation):

    def __init__(self, 
                 W: Tensor):
        super().__init__(W)
    
    def _compute_output(self):
        return torch.mm(self.input_, self.param)
    
    def _compute_grads(self, output_grad):
        return torch.mm(output_grad, self.param.transpose(0, 1))
    
    def _param_grad(self, 
                    output_grad: Tensor):
        
        return torch.mm(self.input_.transpose(0, 1), output_grad)

In [6]:
def assert_same_shape(output: Tensor, 
                      output_grad: Tensor):
    assert output.shape == output_grad.shape, \
    '''
    Two tensors should have the same shape; instead, first Tensor's shape is {0}
    and second Tensor's shape is {1}.
    '''.format(tuple(output_grad.shape), tuple(output.shape))
    return None

## 1D Convolution

1 input, 1 output

## With padding

In [140]:
def _pad_1d(inp: Tensor,
            num: int) -> Tensor:
    z = torch.Tensor([0])
    z = z.repeat(num)
    return torch.cat([z, inp, z])

In [141]:
_pad_1d(inp1, 1)

tensor([0., 0., 1., 2., 3., 4., 5., 6., 0.])

In [142]:
def conv_1d(inp: Tensor, 
            fil: Tensor) -> Tensor:
    
    fil_len = fil.shape[0]
    fil_mid = fil_len // 2
    inp_pad = _pad_1d(inp, fil_mid)
    
    out = torch.zeros(inp.shape)
    
    for o in range(out.shape[0]):
        for f in range(fil_len):
            out[o] += fil[f] * inp_pad[o+f]

    assert_same_shape(inp, out)
    return out

In [117]:
inp1 = Tensor([0,1,2,3,4,5,6])
fil1 = Tensor([1,1,1])
fil2 = Tensor([2,1,1])

In [118]:
conv_1d(inp1, fil1)

tensor([ 1.,  3.,  6.,  9., 12., 15., 11.])

In [ ]:
conv_1d(inp1, fil2)

In [143]:
def conv_1d_sum(inp: Tensor, 
                fil: Tensor) -> Tensor:
    out = conv_1d(inp, fil)
    return torch.sum(out)

In [144]:
conv_1d_sum(inp1, fil1)

tensor(57.)

In [145]:
conv_1d_sum(inp1, fil2)

tensor(72.)

### Gradients

In [146]:
def _param_grad(inp: Tensor, 
                fil: Tensor, 
                output_grad: Tensor = None) -> Tensor:
    
    fil_len = fil.shape[0]
    fil_mid = fil_len // 2
    inp_pad = _pad_1d(inp, fil_mid)
    
    if not output_grad:
        out_grad = torch.ones_like(inp)
    else:
        assert_same_shape(inp, out_grad)
    
    out_pad = _pad_1d(out_grad, fil_mid)
    
    # Zero padded 1 dimensional convolution
    param_grad = torch.zeros_like(fil)
    input_grad = torch.zeros_like(inp)

    for f in range(fil.shape[0]):
        for o in range(inp.shape[0]):
            param_grad[f] += inp_pad[o+f] * out_grad[o]
            input_grad[o] += out_pad[o+fil_len-f-1] * fil[f]
        
    assert_same_shape(param_grad, fil)
    assert_same_shape(input_grad, inp)
    return param_grad, input_grad

### Gradient check

In [147]:
inp_t = Tensor([0,1,2,3,4,5,6])
fil_t = Tensor([1,1,1])

In [148]:
def _check_grad(inp: Tensor,
                fil: Tensor):
    
    inp_grad_check = torch.zeros_like(inp)
    fil_grad_check = torch.zeros_like(fil)

    for i in range(inp.shape[0]):
        inp_temp = inp.clone()
        inp_temp[i] = inp_temp[i] + 1
        sum1 = conv_1d_sum(inp, fil).item()
        sum2 = conv_1d_sum(inp_temp, fil).item()
        inp_grad_check[i] = sum2 - sum1

    for f in range(fil.shape[0]):
        fil_temp = fil.clone()
        fil_temp[f] = fil_temp[f] + 1
        sum1 = conv_1d_sum(inp, fil).item()
        sum2 = conv_1d_sum(inp, fil_temp).item()
        fil_grad_check[f] = sum2 - sum1
        
    return fil_grad_check, inp_grad_check

In [149]:
_param_grad(inp_t, fil_t)

(tensor([15., 21., 21.]), tensor([2., 3., 3., 3., 3., 3., 2.]))

In [150]:
_check_grad(inp_t, fil_t)

(tensor([15., 21., 21.]), tensor([2., 3., 3., 3., 3., 3., 2.]))

## Without padding

In [8]:
def _min_filter(inp: int, 
                fil_mid: int):
    '''
    For Convolution: for an odd filter size and input index, finds the 
    minimum filter index
    '''
    if inp < fil_mid:
        return fil_mid - inp
    else:
        return 0
    
def _max_filter(inp, fil_max, max_inp):
    '''
    For Convolution: for an odd filter size and input index, finds the 
    minimum filter index
    '''
    if max_inp - inp < fil_max:
        return max_inp - inp
    else:
        return fil_max

In [9]:
out = torch.zeros(inp.shape)

In [10]:
inp = Tensor([0,1,2,3,4,5,6])
fil = Tensor([1,1,1])
# out = 1, 3, 6, 9, 12, 15, 11

In [11]:
for j in range(out.shape[0]):
    min_fil = _min_filter(j, fil_mid)
    max_fil = _max_filter(j, fil_max, max_inp)
    for f, i in zip(range(min_fil, max_fil+1), range(min_fil - fil_mid, max_fil - fil_mid+1)):
        out[j] += inp[j+i] * fil[f] 

In [12]:
out

tensor([ 1.,  3.,  6.,  9., 12., 15., 11.])

### Param grad

Loop over the output.

The filter gets multiplied by the same elements in the input.



In [ ]:


for i in range(len(fil)):
    

In [229]:
def Conv1D(inp: Tensor, 
           fil: Tensor) -> Tensor:
    '''
    1D Convolution with Zero padding
    '''
    out = torch.zeros(inp.shape)
    
    for j in range(inp.shape[0]):
        min_fil = _min_filter(j, fil_mid)
        max_fil = _max_filter(j, fil_max, max_inp)
        for f, i in zip(range(min_fil, max_fil+1), range(min_fil - fil_mid, max_fil - fil_mid+1)):
            out[j] += inp[j+i] * fil[f] 
    
    return out

In [ ]:
def Conv1D_param_grad(out_grad: Tensor, 
                      inp: Tensor, 
                      param: Tensor) -> Tensor:
    '''
    1D Convolution with Zero padding
    '''
    param_grad = torch.zeros(param.shape)
    
    for j in range(param.shape[0]):
        for i, ran(max(ind - filter_mid), range(min_fil - fil_mid, max_fil - fil_mid+1)):
    
    return out

In [ ]:
class Conv1D(ParamOperation):

    def __init__(self, 
                 fil: Tensor):
        super().__init__(fil)
    
    def _init_out(self):
        self.output = torch.empty(self.input_.shape)
    
    def _compute_output(self):
        for i in self.input_.shape[0]:
            
    
    def _compute_grads(self, output_grad):
        return torch.mm(output_grad, self.param.transpose(0, 1))
    
    def _param_grad(self, 
                    output_grad: Tensor):
        
        return torch.mm(self.input_.transpose(0, 1), output_grad)